<a href="https://colab.research.google.com/github/periwinklemctartins/Coronavirus--Propogation-Simulation/blob/master/VectorEquations3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
a = torch.zeros(4, dtype = torch.int32)

print(a)

tensor([0, 0, 0, 0], dtype=torch.int32)


In [0]:
a = torch.tensor([1,2])
b = torch.tensor([1,2])

c = a + b

print(c)

tensor([2, 4])


In [0]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

## Variable Initialization

In [140]:
provpop = torch.tensor([4413146, 5110917, 1377517, 779993, 521365, 44904, 977457, 39097, 14711827, 158158, 8537674, 1181666, 41078], dtype = float)

L = torch.tensor([[6,    -1,    -1,     0,     0,     0,     0,     0,    -1,     0,    -1,    -1,    -1], 
                  [-1,    6,    -1,     0,     0,    -1,     0,     0,    -1,     0,    -1,    -1,     0],
                  [-1,    1,     7,     0,     0,    -1,     0,    -1,    -1,     0,    -1,    -1,     0],
                  [0,     0,     0,     4,    -1,     0,     0,     0,    -1,    -1,    -1,     0,     0],
                  [0,     0,     0,    -1,     5,     0,    -1,     0,    -1,    -1,    -1,     0,     0],
                  [0,    -1,    -1,     0,     0,     7,     0,    -1,    -1,     0,    -1,    -1,    -1],
                  [0,     0,     0,     0,    -1,     0,     4,     0,    -1,    -1,    -1,     0,     0],
                  [0,     0,    -1,     0,     0,    -1,     0,     6,    -1,     0,    -1,    -1,    -1],
                  [-1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    12,    -1,    -1,    -1,   -1],
                  [0,     0,     0,    -1,    -1,     0,    -1,     0,    -1,     5,    -1,     0,     0],
                  [-1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    -1,    12,    -1,   -1],
                  [-1,    -1,    -1,     0,     0,    -1,     0,    -1,    -1,     0,    -1,     7,    0],
                  [-1,     0,     0,     0,     0,    -1,     0,    -1,    -1,     0,    -1,     0,    5]], dtype = float);

S =  torch.zeros(13, dtype = torch.float)
E =  torch.zeros(13, dtype = torch.float)
I =  torch.zeros(13, dtype = torch.float)
S0 = torch.zeros(13, dtype = torch.float)
E0 = torch.zeros(13, dtype = torch.float)
I0 = torch.zeros(13, dtype = torch.float)

FE = 0.01     # Fraction of population that is initially exposed

S = provpop*(1 - FE)        # Initializes Susceptible
E = provpop*(1 - FE)        # Initializes Exposed
I = provpop - provpop       # I know this last one looks fucking stupid but it's the only way I get this program to work without shitting itself

dt = 0.05; nt = 4000

alpha = 0.2
beta  = 0.5
gamma = 0.01
mu    = 0.075
kE    = 0.0005
kS    = 0.0005
kI    = 0
theta = torch.tensor([alpha, beta, gamma, mu, kE, kS, kI])

P = ['AB', 'BC', 'MB', 'NB', 'NL', 'NT', 'NS', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']

print(S,E,I*p)

Sout = torch.zeros(13,nt+1, dtype = float)
Eout = torch.zeros(13,nt+1, dtype = float)
Iout = torch.zeros(13,nt+1, dtype = float)

tensor([ 4369014.5400,  5059807.8300,  1363741.8300,   772193.0700,
          516151.3500,    44454.9600,   967682.4300,    38706.0300,
        14564708.7300,   156576.4200,  8452297.2600,  1169849.3400,
           40667.2200], dtype=torch.float64) tensor([ 4369014.5400,  5059807.8300,  1363741.8300,   772193.0700,
          516151.3500,    44454.9600,   967682.4300,    38706.0300,
        14564708.7300,   156576.4200,  8452297.2600,  1169849.3400,
           40667.2200], dtype=torch.float64) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)


## Main Function

In [142]:
Sout, Eout, Iout = integrateSEI(theta,S0,E0,I0,dt,nt,p)

S = torch.zeros(nt+1, dtype = float) 
E = torch.zeros(nt+1, dtype = float)
I = torch.zeros(nt+1, dtype = float)

for n in range(13):
  for i in range(nt+1):
    S[i] = Sout[n,i]; E[i] = Eout[n,i]; I[i] = Iout[n,i]
  
  t = np.arange(nt+1)*dt
  title = 'maximum of infected people %', P[n], # torch.max(I).item()
  plt.figure(n)
  plt.title(title)
  plt.plot(t,S,t,E,t,I)

RuntimeError: ignored

In [0]:
# the numbers it spits out are fractions of the population not total # of people
def SEImodel(theta,S,E,I,p):
    
    dSdt = torch.zeros(13, dtype = float)
    dEdt = torch.zeros(13, dtype = float)
    dIdt = torch.zeros(13, dtype = float)

    alpha = theta[0]; beta  = theta[1]; gamma = theta[2]; mu    = theta[3]; kE    = theta[4]; kS    = theta[5]; kI    = theta[6]

    #Method 1
    dSdt = (-kS*torch.mv(L,S*p))/p - beta*((E/p)*(S/p)) - gamma*((I/p)*(S/p))               
    dEdt = (-kE*torch.mv(L,E*p))/p + beta*(E*S) + gamma*(I*S) - alpha*E     
    dIdt = (-kI*torch.mv(L,I*p))/p + alpha*E - mu*I                      

    #Method 2
    #dSdt  = -kS*torch.mv(L,S*p) - beta*(E*S) - gamma*(I*S)                # dS/dt
    #dEdt  = -kE*torch.mv(L,E*p) + beta*(E*S) + gamma*(I*S) - alpha*E      # dE/dt
    #dIdt  = -kI*torch.mv(L,I*p) + alpha*E - mu*I                       # dI/dt
    
    return dSdt, dEdt, dIdt,

In [0]:
def integrateSEI(theta,S0,E0,I0,dt,nt,p):
    
    # tensors to save the results over time
    Sout = torch.zeros(13,nt+1, dtype = float)
    Eout = torch.zeros(13,nt+1, dtype = float)
    Iout = torch.zeros(13,nt+1, dtype = float)
    dSdt = torch.zeros(13, dtype = float)
    dEdt = torch.zeros(13, dtype = float)
    dIdt = torch.zeros(13, dtype = float)
    
    for i in range(13):
        Sout[i,0] = S0[i]
        Eout[i,0] = E0[i]
        Iout[i,0] = E0[i]
    
    S = S0; E = E0; I = I0
    for i in range(nt):
        
        dSdt, dEdt, dIdt = SEImodel(theta,S,E,I,p)
        S += dt*dSdt
        E += dt*dEdt
        I += dt*dIdt
        
        for n in range(13):
            Sout[n,i+1] = S[n]; Eout[n,i+1] = E[n]; Iout[n,i+1] = I[n]
        
    return Sout, Eout, Iout

# TEST MAIN

In [132]:
a,b,c = SEImodel(theta,S,E,I,p)

print(a,b,c)

tensor([-0.4900, -0.4900, -0.4901, -0.4900, -0.4900, -0.4900, -0.4900, -0.4900,
        -0.4900, -0.4901, -0.4900, -0.4901, -0.4900], dtype=torch.float64) tensor([9.5441e+12, 1.2801e+13, 9.2990e+11, 2.9814e+11, 1.3321e+11, 9.9163e+08,
        4.6820e+11, 7.5278e+08, 1.0607e+14, 1.2259e+10, 3.5721e+13, 6.8427e+11,
        8.3062e+08], dtype=torch.float64) tensor([ 873802.9210, 1011961.5811,  272748.3701,  154438.6163,  103230.2715,
           8890.9921,  193536.4889,    7741.2061, 2912941.7894,   31315.2845,
        1690459.4772,  233969.8715,    8133.4441], dtype=torch.float64)


# JUNK

In [97]:
provpop = torch.tensor([4413146, 5110917, 1377517, 779993, 521365, 44904, 977457, 39097, 14711827, 158158, 8537674, 1181666, 41078])

S = torch.zeros(13, dtype = float)
E = torch.zeros(13, dtype = float)
I = torch.zeros(13, dtype = float)

FE = 0.01     # Fraction of population that is initially exposed

# Initializes Susceptible
for i in range(13):
  S[i] = 1 - FE
  E[i] = FE

alpha = 0.6
beta  = 0.2
gamma = 0.1
mu    = 0.021
kE    = 0.0
kS    = 0
kI    = 0
theta = torch.tensor([alpha, beta, gamma, mu, kE, kS, kI])

# dSdt, dEdt, dIdt = SEImodel(theta,S,E,I,p)

print(S)

tensor([0.9900, 0.9900, 0.9900, 0.9900, 0.9900, 0.9900, 0.9900, 0.9900, 0.9900,
        0.9900, 0.9900, 0.9900, 0.9900], dtype=torch.float64)


In [0]:
# tensors to save the results over time
    Sout = torch.zeros(13,nt+1, dtype = torch.float)
    Eout = torch.zeros(13,nt+1, dtype = torch.float)
    Iout = torch.zeros(13,nt+1, dtype = torch.float)
    S = torch.zeros(13, dtype = torch.float)
    E = torch.zeros(13, dtype = torch.float)
    I = torch.zeros(13, dtype = torch.float)
    
    for i in range(13):
        Sout[i,0] = S0[i]
        Eout[i,0] = E0[i]
        Iout[i,0] = E0[i]

        S[i] = S0[i]; E[i] = E0[i]; I[i] = I0[i]
    
    S = S0[i]; E = E0[i]; I = I0[i]
    for i in range(nt):
        
        dSdt, dEdt, dIdt = SEImodel(theta,S,E,I)
        S += dt*dSdt
        E += dt*dEdt
        I += dt*dIdt
        
        for n in range(13):
            Sout[n,i+1] = S[n]; Eout[n,i+1] = E[n]; Iout[n,i+1] = I[n]

In [99]:
dSdt, dEdt, dIdt = SEImodel(theta,S,E,I,p)

print(dSdt, dEdt, dIdt)

tensor([-0.0020, -0.0020, -0.0020, -0.0020, -0.0020, -0.0020, -0.0020, -0.0020,
        -0.0020, -0.0020, -0.0020, -0.0020, -0.0020], dtype=torch.float64) tensor([-0.0040, -0.0040, -0.0040, -0.0040, -0.0040, -0.0040, -0.0040, -0.0040,
        -0.0040, -0.0040, -0.0040, -0.0040, -0.0040], dtype=torch.float64) tensor([0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060, 0.0060,
        0.0060, 0.0060, 0.0060, 0.0060], dtype=torch.float64)


In [0]:
# province = [abrev, population, # of initial infections, time between first case in Canada and first case in province]

AB = ['AB', 4413146, 1, 37]
BC = ['BC', 5110917, 1, 0]
MB = ['MB', 1377517, 1, 42]
NB = ['NB', 779993, 1, 42]
NL = ['NL', 521365, 1, 48]
NT = ['NT', 44904, 1, 50]
NS = ['NS', 977457, 1, 47]
NU = ['NU', 39097, 0, 0]
ON = ['ON', 14711827, 3, 0]
PE = ['PE', 158158, 1, 44]
QC = ['QC', 8537674, 1, 30]
SK = ['SK', 1181666, 1, 45]
YT = ['YT', 41078, 2, 52]

Provinces = [ AB, BC, MB, NB, NL, NT, NS, NU, ON, PE, QC, SK, YT ]